In [1]:
import cutlass
import cutlass.cute as cute

In [2]:
@cute.kernel 
def kernel(): 
  tidx, _, _ = cute.arch.thread_idx()
  if tidx == 0:
    cute.printf("Hello World!")

In [3]:
@cute.jit
def hello_world():
    # Print hello world from host code
    cute.printf("hello world")

    # Launch kernel
    kernel().launch(
        grid=(1, 1, 1),  # Single thread block
        block=(32, 1, 1),  # One warp (32 threads) per thread block
    )

In [ ]:
cutlass.cuda.initialize_cuda_context()

# Method 1: Just-In-Time (JIT) compilation - compiles and runs the code immediately
print("Running hello_world()...")
hello_world()

# Method 2: Compile first (useful if you want to run the same code multiple times)
print("Compiling...")
hello_world_compiled = cute.compile(hello_world)

# Dump PTX/CUBIN files while compiling
from cutlass.cute import KeepPTX, KeepCUBIN

print("Compiling with PTX/CUBIN dumped...")
# Alternatively, compile with string based options like
# cute.compile(hello_world, options="--keep-ptx --keep-cubin") would also work.
hello_world_compiled_ptx_on = cute.compile[KeepPTX, KeepCUBIN](hello_world)

# Run the pre-compiled version
print("Running compiled version...")
hello_world_compiled()

Running hello_world()...
hello world
Compiling...
Hello World!
Compiling with PTX/CUBIN dumped...
Running compiled version...
hello world


0

Hello World!


In [26]:
@cute.jit
def make_layout():
  layout = cute.make_layout((3,4),stride=(1,3))
  cute.printf(cute.coalesce(layout))
  for i in range(cute.size(layout)):
    cute.printf(layout(i)) 
    
  return layout

In [27]:
L = make_layout()

12:1
0
1
2
3
4
5
6
7
8
9
10
11
